In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import scipy as sp
import sklearn as sk # data mining tools
import matplotlib.pylab as plt # plotting
import seaborn as sns # advanced plotting
from pandas.tools.plotting import scatter_matrix
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

In [2]:
winemag = pd.read_csv("data_varietytfIdf_stemmedData.csv")

In [3]:
useless_columns=['Unnamed: 0','taster_twitter_handle','region_2']

### This function below performs a one-hot-encoding... unfortunatly it must be executed for each column because it runs out of memory!

### Second Dataset: 
               columns removed -> 'Unnamed: 0','description,'taster_twitter_handle','region_2','title'
               columns added -> 'vintage'
               dropped duplicates basing on -> "description","price","points","taster_name","title"
               missing values for price -> replaced with mean prices from that winery
               missing values on the others features -> replaced with "unknown"
               columns encoded with one-hot-encoding---> beware, not able to encode designation,title and winery!!

In [ ]:
def cleanDataset_2(dataset,columns_to_drop):
    df = dataset
    df = df.drop(columns_to_drop,axis=1)
    df = df[df.country.isnull()==False]
    df = df[df.province.isnull()==False]
    df = df[df.variety.isnull()==False]
    df = df.drop_duplicates(["description","price","points","taster_name","title"])
    #avgPriceWinery = df.groupby("winery")#.price.mean()
    #df['price'] = df.apply(lambda row: avgPriceWinery.get_group(row["winery"]).price.mean()
    #                       if pd.isnull(row['price']) else row['price'],axis=1)
    #df["price"].fillna(df["price"].mean(),inplace=True) <-- policy of filling with the global mean

    #prende solo recensioni con autore conosciuto, ha senso come cosa?
    #data.dropna(subset=["taster_name"],inplace=True)
    df = df.drop('description',axis=1)
    #df.price.fillna(0,inplace=True)
    #df['price']=df['price'].astype('int64')
    df = df[df.price.isnull()==False]
    df.fillna('unknown',inplace=True)
    #df = df[df.price.isnull()==False]

    return df

In [4]:
#df_2 = cleanDataset_2(winemag,useless_columns)
winemag.isnull().sum()
df_2=winemag

In [5]:
import re
pattern = r"\d{4}"
vintages = []
for elem in df_2.title:
    match = re.findall(pattern,elem)
    year = 0
    if len(match)>1:
        y= int(match[0])
        if y <1952:
            year = 0
        elif y==3000:
            year == 2009
        elif y == 7200:
            year = int(match[1])
        else:
            year= y
    elif len(match)==1:
        y= int(match[0])
        if y <1952:
            year = 0
        elif y==3000:
            year == 2009
        elif year == 7200:
            year = 0
        else:
            year = y
    else:
        year = 0
    vintages.append(year)
se = pd.Series(vintages)
df_2['vintage']=se.values 
df_2.drop('title',axis=1)

,country,description,designation,points,price,province,region_1,taster_name,variety,winery,description_tfIdf,vintage
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,Kerin O’Keefe,White Blend,Nicosia,"[0.04885833490039738, 0.3057715203598046, 0.38...",2013
1,Portugal,fruiti firm juici red berri drinkabl,Avidagos,87,15.000000,Douro,unknown,Roger Voss,Portuguese Red,Quinta dos Avidagos,"[0.03228609409338993, 0.03883847038764645, 0.0...",2011
2,US,tart snappi lime flesh domin green pineappl po...,unknown,87,14.000000,Oregon,Willamette Valley,Paul Gregutt,Pinot Gris,Rainstorm,"[0.019578555849338562, 0.03787011890072908, 0....",2013
3,US,pineappl rind lemon pith orang blossom opul no...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Alexander Peartree,Riesling,St. Julian,"[0.09546425350974236, 0.20805243488881586, 0.0...",2013
4,US,much regular bottl rough tannic rustic earthi ...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Paul Gregutt,Pinot Noir,Sweet Cheeks,"[0.1711900090314572, 0.3208454187539189, 0.232...",2012
5,Spain,blackberri typic navarran whiff green herb cas...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,Michael Schachner,Tempranillo-Merlot,Tandem,"[6.411000151830199e-05, 0.0009691491284567166,...",2011
6,Italy,bright inform candi berri white pepper savori ...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Frappato,Terre di Giurfo,"[0.0006457325944628814, 0.007391639311866063, ...",2013
7,France,dri restrain offer profus firm textur much food,unknown,87,24.000000,Alsace,Alsace,Roger Voss,Gewürztraminer,Trimbach,"[0.008169774845361692, 0.06847392686501436, 0....",2012
8,Germany,savori thyme note accent preserv peach brisk o...,Shine,87,12.000000,Rheinhessen,unknown,Anna Lee C. Iijima,Gewürztraminer,Heinz Eifel,"[0.006270635180525164, 0.0032548866092490802, ...",2013
9,France,great depth fresh appl pear touch spice dri cr...,Les Natures,87,27.000000,Alsace,Alsace,Roger Voss,Pinot Gris,Jean-Baptiste Adam,"[0.015969532708426708, 0.02142619849953519, 0....",2012


In [6]:

######## FUNCTION TO ENCODE ######## 
def df_column_Encode(df_in,column):
    df = df_in
    col = column
    if type(df[col][1])==str: # this time I had to put the first value to catch the type becouse removing rows with->
        #-> price null I removed the first row.. and so pands didn't recognize the first row as 0..
        enc = preprocessing.LabelEncoder()
        feature_labeled=enc.fit_transform(df[col]) #need to give to OneHotEncoder a column of int's
        feature_labeled = feature_labeled.reshape(-1, 1)# need to reshape
        ohe = preprocessing.OneHotEncoder(sparse=True) #true
        final = ohe.fit_transform(feature_labeled)
        print(list(final.toarray()[0]))
        df[col] = str(list(final.toarray()[0]))
    return df

In [7]:
from ast import literal_eval

#description with values may cause trouble
descTop = []
for idx, row in df_2.iterrows():
    el = literal_eval(row["description_tfIdf"])  #max(el) with this function doesn't work so i made it myself
    m = 0
    for i in el:
        if(i > m):
            m = i
    descTop.append(m)
df_2["max"] = descTop
df_2

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,description_tfIdf,vintage,max
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,"[0.04885833490039738, 0.3057715203598046, 0.38...",2013,0.384445
1,Portugal,fruiti firm juici red berri drinkabl,Avidagos,87,15.000000,Douro,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,"[0.03228609409338993, 0.03883847038764645, 0.0...",2011,0.076411
2,US,tart snappi lime flesh domin green pineappl po...,unknown,87,14.000000,Oregon,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,"[0.019578555849338562, 0.03787011890072908, 0....",2013,0.207796
3,US,pineappl rind lemon pith orang blossom opul no...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,"[0.09546425350974236, 0.20805243488881586, 0.0...",2013,1.425857
4,US,much regular bottl rough tannic rustic earthi ...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,"[0.1711900090314572, 0.3208454187539189, 0.232...",2012,0.741792
5,Spain,blackberri typic navarran whiff green herb cas...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,"[6.411000151830199e-05, 0.0009691491284567166,...",2011,0.040243
6,Italy,bright inform candi berri white pepper savori ...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,"[0.0006457325944628814, 0.007391639311866063, ...",2013,0.007392
7,France,dri restrain offer profus firm textur much food,unknown,87,24.000000,Alsace,Alsace,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,"[0.008169774845361692, 0.06847392686501436, 0....",2012,0.156063
8,Germany,savori thyme note accent preserv peach brisk o...,Shine,87,12.000000,Rheinhessen,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,"[0.006270635180525164, 0.0032548866092490802, ...",2013,0.532834
9,France,great depth fresh appl pear touch spice dri cr...,Les Natures,87,27.000000,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,"[0.015969532708426708, 0.02142619849953519, 0....",2012,0.026868


In [8]:
df_e_2= df_column_Encode(df_2,'country')
df_e_2= df_column_Encode(df_2,'province')
df_e_2= df_column_Encode(df_2,'region_1')
df_e_2= df_column_Encode(df_2,'taster_name')
df_e_2= df_column_Encode(df_2,'variety')
df_2

#df_e= df_column_Encode(df_2,'designation') ---> impossible.. it runs out of memory!!!
#df_e_2= df_column_Encode(df_2,'points')
#df_e_2= df_column_Encode(df_2,'price')
#df_e_2= df_column_Encode(df_2,'winery') -----> ..
#df_e_2= df_column_Encode(df_2,'vintage')

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,description_tfIdf,vintage,max
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Nicosia 2013 Vulkà Bianco (Etna),"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Nicosia,"[0.04885833490039738, 0.3057715203598046, 0.38...",2013,0.384445
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",fruiti firm juici red berri drinkabl,Avidagos,87,15.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Quinta dos Avidagos 2011 Avidagos Red (Douro),"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Quinta dos Avidagos,"[0.03228609409338993, 0.03883847038764645, 0.0...",2011,0.076411
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",tart snappi lime flesh domin green pineappl po...,unknown,87,14.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Rainstorm 2013 Pinot Gris (Willamette Valley),"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Rainstorm,"[0.019578555849338562, 0.03787011890072908, 0....",2013,0.207796
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",pineappl rind lemon pith orang blossom opul no...,Reserve Late Harvest,87,13.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",St. Julian 2013 Reserve Late Harvest Riesling ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",St. Julian,"[0.09546425350974236, 0.20805243488881586, 0.0...",2013,1.425857
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",much regular bottl rough tannic rustic earthi ...,Vintner's Reserve Wild Child Block,87,65.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Sweet Cheeks,"[0.1711900090314572, 0.3208454187539189, 0.232...",2012,0.741792
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",blackberri typic navarran whiff green herb cas...,Ars In Vitro,87,15.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Tandem,"[6.411000151830199e-05, 0.0009691491284567166,...",2011,0.040243
6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",bright inform candi berri white pepper savori ...,Belsito,87,16.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Terre di Giurfo 2013 Belsito Frappato (Vittoria),"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Terre di Giurfo,"[0.0006457325944628814, 0.007391639311866063, ...",2013,0.007392
7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",dri restrain offer profus firm textur much food,unknown,87,24.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",Trimbach 2012 Gewurztraminer (Alsace),"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Trimbach,"[0.008169774845361692, 0.06847392686501436, 0....",2012,0.156063
8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",savori thyme note accent preserv peach brisk o...,Shine,87,12.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.

In [9]:
df_2.drop(["description","description_tfIdf"],axis=1,inplace=True)

df_2.to_csv('test1.csv', sep=',', encoding='utf-8')

In [10]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117651 entries, 0 to 117650
Data columns (total 12 columns):
country        117651 non-null object
designation    117651 non-null object
points         117651 non-null int64
price          117651 non-null float64
province       117651 non-null object
region_1       117651 non-null object
taster_name    117651 non-null object
title          117651 non-null object
variety        117651 non-null object
winery         117651 non-null object
vintage        117651 non-null int64
max            117651 non-null float64
dtypes: float64(2), int64(2), object(8)
memory usage: 10.8+ MB


In [ ]:
def df_column_Decode(df,column):
    pass

... still working on inverse transformation.. for the work after classification... see link[https://stackoverflow.com/questions/22548731/how-to-reverse-sklearn-onehotencoder-transform-to-recover-original-data]